# Running a Random Forest on A Larger Dataset

In [1]:
from SEData.data import *
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from time import sleep
from random import randint

In [5]:
questions = pd.read_csv('most_recent_by_category.csv', squeeze = True, index_col=0)

In [20]:
n = 10000 # Size of data set
l = round(n/ len(questions)) #Will create l links for each category. Also ensures balanced labels.

278

In [6]:
data_links = back_generate_links(questions['url'], l)


['https://worldbuilding.stackexchange.com/questions/82773',
 'https://worldbuilding.stackexchange.com/questions/82772',
 'https://worldbuilding.stackexchange.com/questions/82771',
 'https://worldbuilding.stackexchange.com/questions/82770',
 'https://worldbuilding.stackexchange.com/questions/82769',
 'https://worldbuilding.stackexchange.com/questions/82768',
 'https://worldbuilding.stackexchange.com/questions/82767',
 'https://worldbuilding.stackexchange.com/questions/82766',
 'https://worldbuilding.stackexchange.com/questions/82765',
 'https://worldbuilding.stackexchange.com/questions/82764']

In [4]:
data = pd.DataFrame([get_text(link, pause = True) for link in data_links])


ConnectionError: HTTPSConnectionPool(host='academia.stackexchange.com', port=443): Max retries exceeded with url: /questions/96273 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x0000026A6DA01EF0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [ ]:
data.dropna(axis = 0, how = 'any', inplace = True) #Drop NAs in place.
data.columns = ['labels','text']

In [ ]:
train_data, test_data = train_test_split(data, test_size = .25) #stratify = data['labels'])

In [ ]:
train_corpus = train_data['text']
train_labels = train_data['labels']
test_corpus = test_data['text']
test_labels = test_data['labels']

In [ ]:
tfidf = TfidfVectorizer(stop_words = 'english', max_features = 500) #Limit number of features at 500 words with highest Tfidf score
train_matrix = tfidf.fit_transform(train_corpus)

In [ ]:
test_matrix = tfidf.transform(test_corpus)


## Train a Decision Tree Model
### X_train : train_matrix
### y_train : train_labels
### X_test : test_matrix
### y_test : test_labels

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest.fit(train_matrix, train_labels)

pred_labels = forest.predict(test_matrix)

In [ ]:
print(accuracy_score(test_labels, pred_labels))

In [ ]:
pd.DataFrame({"50 Test Data Labels":test_labels[:50], "50 Predicted Data Labels":pred_labels[:50]})